In [1]:
from libs.pipeline_utils import build_sampler_env, sample

In [2]:
batch_size = 64
enc_seq_len = 64
dec_seq_len = 201

In [3]:
load_dir = "./save/shm_c1"

In [4]:
sampler_args = build_sampler_env(load_dir, batch_size, enc_seq_len, dec_seq_len)

INFO:tensorflow:Restoring parameters from ./save/shm_c1/model.ckpt-58519
INFO:tensorflow:Restoring parameters from ./save/shm_c1/model.ckpt-58519


In [12]:
seed_phrase = 'Здравствуйте, как ваше здоровье?'
n_items = 10000

In [6]:
from libs.utils import load_dictionary, load_transformer

In [8]:
dictionary = load_dictionary("./data/")
transformer = load_transformer(load_dir)

In [13]:
%%time
sampled = sample(*sampler_args, 
                 dictionary=dictionary, 
                 transformer=transformer, 
                 seed_phrase=seed_phrase, 
                 n_items=n_items,
                 batch_size=batch_size)

CPU times: user 2h 16min 59s, sys: 5min 20s, total: 2h 22min 19s
Wall time: 28min 15s


In [26]:
sampled[:10]

['так я теперь все узнаю да мне то, что вы мне нужны, коль я угадаю.',
 'позвольтес, сидите исследовать: да, боже мой!',
 'лицо его выражало чемто веселым.',
 'сорока моя тетушка!',
 'идиот, поля!',
 'рассказывал я нам,  но князь немного вскинул плечами.',
 'он не отвечал, настенька, мне в дом не было оставлять в прекрасные слова.',
 'известно, что лучше себя не бежать.',
 'я  ответчик он мог всегда только вытащить.',
 'позвольте мне уйти.']

In [15]:
from libs.pipeline_utils import wrap_list_with_score

In [27]:
wrapped = wrap_list_with_score(sampled)

In [17]:
from libs.utils import top_best

In [28]:
top_best(wrap_list_with_score(sampled), 0.0002)

[('так я теперь все узнаю да мне то, что вы мне нужны, коль я угадаю.', 1.0),
 ('позвольтес, сидите исследовать: да, боже мой!', 1.0)]

In [32]:
from libs.utils import load_dis

dis = load_dis('./models/', 'believability')

In [34]:
from libs.pipeline_utils import simple_probability_pipeline, probability

In [35]:
from functools import partial

In [36]:
be_f = partial(probability, model=dis, transformer=transformer)

In [37]:
sample_f = partial(sample, *sampler_args, 
                   dictionary=dictionary, 
                   transformer=transformer, 
                   seed_phrase=seed_phrase, 
                   n_items=100,
                   batch_size=64)

In [39]:
simple_probability_pipeline(sample_f, [be_f, be_f], topn=0.2)

[('смотри!', 0.53956571538026665),
 ('должно быть!', 0.54014616695964435),
 ('господи боже мой!', 0.54020416794833181),
 ('и как так запретили!', 0.54057923438085709)]